TAREA 1: Visualizacion de datos 2024-1 
- Nombre: Nicolas Esteban Parra Garcia  
- Matricula: 2019422588\
Trabajo realizado con plotly por que se ve bonito, utilizando documentacion de plotly y referencias de trabajo semanales para manejo de datos

# Setup de Globales
**IMPORTANTE**  ⬇️

In [56]:
# Modificar valor para cantidad en un año determinado (usando un INT) entre 2005 y 2016 , o utilizar el STRING "Total" para la cantidad total
YEAR_VIS_OPTION = "Total"

# Load data


imports de modulos y  biblotecas

In [57]:
import numpy as np  
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Importar el dataframe 

In [58]:
df_chi = pd.read_excel('MigrantesChile (2005-2016).xlsx',
                      sheet_name='Migrantes',
                      skipfooter=3)

Quitar columnas de 'ID Continent' e 'ID Country'

In [59]:
df_chi.drop(['ID Continent','ID Country'], axis=1, inplace=True)

Agregar columna de total, que es la suma de las columnas dtype numero

In [60]:
df_chi['Total'] = df_chi.select_dtypes(include=['number']).sum(axis=1)

# Pregunta 1
Un lineplot con la migración desde los 5 países que aportaron más inmigrantes de su respectivo continente. Se puede considerar, para ello, el aporte total o el de un año determinado.

In [61]:
# df_sorted = filas ordenadas por continente y luego por total de migrantes, separando en grupos por continente, luego se obtiene el primero de cada continente
df_sorted = df_chi.sort_values(by=['Continent','Total'], ascending = False).groupby('Continent').head(1)

In [62]:
#Se quita la columna total para que existan rangos relevantes en el eje y (Migrants) del grafico 
df = df_sorted
df.drop(['Total'], axis=1, inplace=True)
# df_melted almacena df_sorted transformado de un formato amplio (con muchas columnas de años) a un formato largo (con una columna de años y una de valor) para graficar con px.line
df_melted = df.melt(id_vars=["Continent", "Country"], var_name="Year", value_name="Migrants")

In [63]:
# Crear grafico
fig = px.line(df_melted, x='Year', y='Migrants', color='Country', title='Countries with most migrants in Chile by continent')
#Setear los años como valor unico para que se muestren todos en el eje x
all_years = df_melted['Year'].unique()
fig.update_xaxes(tickmode='array', tickvals=all_years)

fig.show()

# Pregunta 2
Dos area plots para la misma consulta de 1, uno apilado y otro no apilado.

In [64]:
# DISCLAIMER: en plotly no se puede hacer un grafico de lineas sin apilar con px.area por lo que se uttiliza go.scatter modificado para crearlo 
#Crear grafico de areas
fig = go.Figure()
for country in df_melted['Country'].unique():
    # filtrar data para cada pais, se utiliza el mismo data frame que la pregunta anterior
    country_data = df_melted[df_melted['Country'] == country]
    # Agregar area para cada pais
    fig.add_trace(go.Scatter(
        x=country_data['Year'],
        y=country_data['Migrants'],
        fill='tozeroy',  # area fill
        mode='lines',  # modo lineas para que sean visibles  
    ))

#Setear los años como valor unico para que se muestren todos en el eje x
fig.update_xaxes(tickmode='array', tickvals=all_years)

fig.update_layout(
    title='Countries with most migrants in Chile by continent (Non-Stacked Areas)',
    xaxis_title='Year',
    yaxis_title='Migrants',
    legend_title='Country'
)

fig.show()

In [65]:


#Crear grafico de areas apiladas
fig = px.area(df_melted, x="Year", y="Migrants", color="Country", line_group="Country",  pattern_shape_sequence=[".", "x", "+"])

fig.update_layout(
    title='Countries with most migrants in Chile by continent (Stacked Areas)',
    xaxis_title='Year',
    yaxis_title='Migrants',
    legend_title='Country'
)
#Setear los años como valor unico para que se muestren todos en el eje x
fig.update_xaxes(tickmode='array', tickvals=all_years)
fig.show()

# Pregunta 3
Cinco histogramas de migrantes por país: un histograma por continente, con cantidad total o cantidad en un año determinado. Importante: deben considerar ajuste de bins.

In [66]:
# se obtienen los nombres de los continentes
continents = df_chi['Continent'].unique()
# subplot por continente
fig = make_subplots(rows=len(continents), cols=1, subplot_titles=continents)

for i, continent in enumerate(continents, start=1):
    # continent_data = data filtrada por continente, (ordenar por claridad)
    continent_data = df_chi[df_chi['Continent'] == continent].sort_values(by=YEAR_VIS_OPTION, ascending=False)
    
    # Se agrega un grafico de barras por continente
    fig.add_trace(
        go.Bar(
            x=continent_data['Country'], 
            y=continent_data[YEAR_VIS_OPTION], 
            name=f'Migrants in {continent}'
        ),
        row=i,
        col=1
    )

# Ajustes al layout para mejorar la visualización
fig.update_layout(
    title_text='Migrants by Country per Continent: ' + str(YEAR_VIS_OPTION),
    height=300 * len(continents),  
    showlegend=False
)
fig.update_xaxes(title_text='Country')
fig.update_yaxes(title_text='Number of Migrants')

fig.show()


# Pregunta 4
Un Bar chart para comparar los migrantes de dos continentes, en cada año considerado en el dataset.

In [67]:
# Continentes a comparar
# Opciones: 'Africa', 'Asia', 'Europe', 'Oceania', 'America', 'Otros'
cont_1 = 'Europe'
cont_2 = 'Asia'

# Filtrar el DataFrame para incluir solo los continentes seleccionados
df_filtered = df_chi[df_chi['Continent'].isin([cont_1, cont_2])]

# Agrupar por año y continente, sumando las cantidades de migrantes
df_grouped = df_filtered.groupby(['Continent']).sum().transpose()
# Remover fila de total y de pais
df_grouped.drop(index='Country', inplace=True)
df_grouped.drop(index='Total', inplace=True)

# Crear el gráfico de barras
fig = go.Figure()
# Agregar las barras para cada continente
for continent in [cont_1, cont_2]:
    fig.add_trace(go.Bar(
        x=df_grouped.index,
        y=df_grouped[continent],
        name=continent
    ))

fig.update_layout(
    title=f'Comparative Migration between {cont_1} and {cont_2} by Year',
    xaxis_title='Year',
    yaxis_title='Number of Migrants',
    barmode='group'  
)

#Setear los años como valor unico para que se muestren todos en el eje x
fig.update_xaxes(tickmode='array', tickvals=all_years)

fig.show()



# Pregunta 5
Un Pie chart de migrantes desde los 5 países europeos que aportan más inmigrantes, con cantidad total o cantidad en un año determinado.

In [68]:
# Extraer los 5 países con más migrantes totales de cada continente
df_sorted = df_chi.sort_values(by=['Continent','Total'], ascending = False).groupby('Continent').head(5)
# Filtrar solo los países de Europa
df_europe = df_sorted[df_sorted['Continent'] == 'Europe']
# Cocinar torta 
fig = go.Figure(data=[go.Pie(
    labels=df_europe['Country'],  
    values=df_europe[YEAR_VIS_OPTION],  
)])

fig.update_layout(
    title=f"Distribution of European Migration to chile : {YEAR_VIS_OPTION}",
)

fig.show()

# Pregunta 6
Un box-and-whisker chart  con los tres países que más inmigrantes aportan, en total o en un año determinado.

In [69]:
# se ordena por el año deseado y se toman los 3 primeros
df_sorted = df_chi.sort_values(by=[YEAR_VIS_OPTION], ascending=False).head(3)

# melt para que los años esten en una columna y sus valores correspondientes en otra
df_melted = df_sorted.melt(id_vars=['Country'], value_vars=all_years,
                           var_name='Year', value_name='Migrants')

# Creacion del plot
fig = px.box(df_melted, x='Country', y='Migrants', color='Country',
             title=f'Top 3 Countries with Most Migrants in Chile: {YEAR_VIS_OPTION}')
fig.show()
